在 AKS 或 Arc Kubernetes 群集上部署 Azure 机器学习扩展后，可以将 Kubernetes 群集附加到 Azure 机器学习工作区，并创建要供 ML 专业人员使用的计算目标。

## 先决条件

将 Kubernetes 群集附加到 Azure 机器学习工作区可以灵活地支持许多不同的方案。 例如，具有多个附件的共享方案、访问 Azure 资源的模型训练脚本以及工作区的身份验证配置。

#### 多次附加和工作负载隔离

一个群集到一个工作区，创建多个计算目标

- 对于同一 Kubernetes 群集，可以多次将其附加到同一工作区，并为不同项目/团队/工作负载创建多个计算目标。

一个群集到多个工作区

- 对于同一 Kubernetes 群集，还可以将其附加到多个工作区，并且多个工作区可以共享同一 Kubernetes 群集。

如果计划针对不同的项目/团队设置不同的计算目标，可以在群集中为计算目标指定现有的 Kubernetes 命名空间，以在不同的团队/项目之间隔离工作负载。

>
> 应事先在群集中创建计划在将群集附加到 Azure 机器学习工作区时指定的命名空间。

#### 从训练脚本安全访问 Azure 资源

如果需要从训练脚本安全访问 Azure 资源，可以在执行附加操作期间为 Kubernetes 计算目标指定[托管标识](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-identity-based-service-authentication?view=azureml-api-2)。

#### 附加到具有用户分配的托管标识的工作区

Azure 机器学习工作区默认使用系统分配的托管标识来访问 Azure 机器学习资源。 如果系统分配的默认设置已打开，则这些步骤已完成。

否则，如果[在创建 Azure 机器学习工作区时指定了用户分配的托管标识](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-identity-based-service-authentication?view=azureml-api-2#user-assigned-managed-identity)，则需要在附加计算之前手动将以下角色分配授予该托管标识。


|Azure 资源名称|要分配的角色|说明|
|---|---|---|
|Azure 中继|Azure 中继所有者|仅适用于启用了 Arc 的 Kubernetes 群集。 如果未连接 Arc，则不会为 AKS 群集创建 Azure 中继。|
|Kubernetes - Azure Arc 或 Azure Kubernetes 服务|读者  <br>Kubernetes 扩展参与者  <br>Azure Kubernetes 服务群集管理员|适用于已启用 Arc 的 Kubernetes 群集和 AKS 群集。|
|Azure Kubernetes 服务|参与者|仅对于使用受信任访问功能的 AKS 群集是必需的。 该工作区使用用户分配的托管标识。 有关详细信息，请参阅[对具有特殊配置的 AKS 集群的 AzureML 访问权限](https://github.com/Azure/AML-Kubernetes/blob/master/docs/azureml-aks-ta-support.md)。|

 提示

部署扩展期间，在与已启用 Arc 的 Kubernetes 群集相同的“资源组”中创建 Azure 中继资源。

 备注

- 如果“Kubernetes 扩展参与者”角色权限不可用，则群集附加失败并出现“未安装扩展”错误。
- 如果“Azure Kubernetes 服务集群管理员”角色权限不可用，则群集附加失败并出现“内部服务器”错误。


## 如何将 Kubernetes 群集附加到 Azure 机器学习工作区

我们支持使用 Azure CLI 或工作室 UI 将 Kubernetes 群集附加到 Azure 机器学习工作区这两种方式。

- [Azure CLI](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-attach-kubernetes-to-workspace?view=azureml-api-2&tabs=cli#tabpanel_1_cli)
- [工作室](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-attach-kubernetes-to-workspace?view=azureml-api-2&tabs=cli#tabpanel_1_studio)
- [Azure SDK](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-attach-kubernetes-to-workspace?view=azureml-api-2&tabs=cli#tabpanel_1_sdk)

以下 CLI v2 命令演示如何附加 AKS 和已启用 Azure Arc 的 Kubernetes 群集，并将其用作启用了托管标识的计算目标。

**AKS 群集**

In [ ]:
extension_name="text-aks-ex"
cluster_name="text-aks-cluster"
resource_group="text-aks-rg"
workspace="text-ml-ws"
subscribe_id =""


In [ ]:
aks_namespace=""
resource_id=f"/subscriptions/{subscribe_id}/resourceGroups/{resource_group}/providers/Microsoft.ContainerService/managedclusters/{cluster_name}"

In [ ]:
!az ml compute attach --resource-group {resource_group} --workspace-name {workspace} --type Kubernetes --name k8s-compute --resource-id  --identity-type SystemAssigned --namespace {aks_namespace} --no-wait

**Arc Kubernetes 群集**

In [ ]:
resource_id=f"/subscriptions/{subscribe_id}/resourceGroups/{resource_group}/providers/Microsoft.Kubernetes/connectedClusters/{cluster_name}"
identity_name=""

user_assigned_identities=f"subscriptions/{subscribe_id}/resourceGroups/{resource_group}/providers/Microsoft.ManagedIdentity/userAssignedIdentities/{identity_name}"

In [ ]:
!az ml compute attach --resource-group {resource_group} --workspace-name {workspace} --type Kubernetes --name amlarc-compute --resource-id {resource_id} --user-assigned-identities {user_assigned_identities} --no-wait

将 `--type` 参数设置为 `Kubernetes`。 使用 `identity_type` 参数启用 `SystemAssigned` 或 `UserAssigned` 托管标识。

 重要

仅 `--user-assigned-identities` 托管标识需要 `UserAssigned`。 虽然可以提供以逗号分隔的用户托管的标识列表，但在附加群集时仅使用第一个标识。

计算附加不会自动创建 Kubernetes 命名空间，也不会验证 kubernetes 命名空间是否存在。 你需要验证群集中是否存在指定的命名空间，否则提交到此计算的任何 Azure 机器学习工作负载都将失败。

## 将托管标识分配给计算目标

开发人员面临的一个共同挑战是如何管理密码和凭据，以确保解决方案的不同组件之间的通信安全。 [托管标识](https://learn.microsoft.com/zh-cn/azure/active-directory/managed-identities-azure-resources/overview)使开发人员无需管理凭据。

若要访问用于 Docker 映像的 Azure 容器注册表 (ACR)，以及用于训练数据的存储帐户，请在启用系统分配或用户分配的托管标识的情况下附加 Kubernetes 计算。

### 分配托管标识

- 可以在计算附加步骤中将托管标识分配给计算。
    
- 如果已附加计算，则可以更新设置以在 Azure 机器学习工作室中使用托管标识。
    
    - 转到 [Azure 机器学习工作室](https://ml.azure.com/)。 选择“计算”、“附加的计算”，然后选择附加的计算。
    - 选择铅笔图标以编辑托管标识。

    
    ![从 Azure 门户更新 Kubernetes 计算标识的屏幕截图。](https://learn.microsoft.com/zh-cn/azure/machine-learning/media/how-to-attach-kubernetes-to-workspace/edit-identity.png?view=azureml-api-2)
    
    ![从 Azure 门户选择 Kubernetes 计算标识的屏幕截图。](https://learn.microsoft.com/zh-cn/azure/machine-learning/media/how-to-attach-kubernetes-to-workspace/update-identity-2.png?view=azureml-api-2)
    

### 向托管标识分配 Azure 角色

Azure 提供了几种向托管标识分配角色的方法。

- [使用 Azure 门户分配角色](https://learn.microsoft.com/zh-cn/azure/role-based-access-control/role-assignments-portal)
- [使用 Azure CLI 分配角色](https://learn.microsoft.com/zh-cn/azure/role-based-access-control/role-assignments-cli)
- [使用 Azure PowerShell 分配角色](https://learn.microsoft.com/zh-cn/azure/role-based-access-control/role-assignments-powershell)

如果使用 Azure 门户分配角色并具有**系统分配的托管标识**，**请选择“用户”**、“**组主体**”或“**服务主体**”，可以通过选择“**选择成员**”来搜索标识名称。 标识名称的格式需要设置为 `<workspace name>/computes/<compute target name>`。

如果具有用户分配的托管标识，请选择“托管标识”以查找目标标识。

可使用托管标识从 Azure 容器注册表拉取映像。 向计算托管标识授予 AcrPull 角色。 有关详细信息，请参阅 [Azure 容器注册表角色和权限](https://learn.microsoft.com/zh-cn/azure/container-registry/container-registry-roles)。

可使用托管标识访问 Azure Blob：

- 出于只读目的，应向计算托管标识授予“存储 Blob 数据读取者”角色。
- 出于读写目的，应向计算托管标识授予“存储 Blob 数据参与者”角色。

## 后续步骤

- [创建和管理实例类型](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-manage-kubernetes-instance-types?view=azureml-api-2)
- [Azure 机器学习推理路由器和连接要求](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-kubernetes-inference-routing-azureml-fe?view=azureml-api-2)
- [保护 AKS 推理环境](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-secure-kubernetes-inferencing-environment?view=azureml-api-2)